# Generate pressure coefficients

The following steps are performed to obtain all the artifacts for pressure coefficient use case:

1. Read parameters file
2. Read Mesh description
   * Its path can be specified
   * Or it can be obtained from the simulation configuration file
3. Read data
   * For reference static pressure
   * For body pressure data
   * Slice from timestep range specified
4. Transform pressure data into pressure coefficient
5. Calculate pressure coefficient statistics
6. Merge coefficient statistics with mesh description into VTK
  
  

Initialize Path Manager

In [4]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager, copy_input_artifacts
import pathlib

path_manager = CpPathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Cp config file

In [5]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)

post_proc_cfg

CpCaseConfig(pressure_coefficient={'default': CpConfig(number_of_chunks=10, timestep_range=(10000.0, 20000.0), reference_pressure='average', U_H=0.05, U_H_correction_factor=1, statistics=['min', 'max', 'std', 'mean', 'skewness', 'kurtosis', 'xtr_min', 'xtr_max', 'mean_qs'])}, extreme_values=ExtremeValuesParameters(CST_real=0.56, CST_sim=16.89, t=0.3, T0=6.0, T1=60.0, yR=1.4, time_scale_correction_factor=0.48))

Read LNAS Mesh

In [6]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Read hist series

In [7]:
import pandas as pd

static_data_path = pathlib.Path(
    "./fixtures/tests/pressure/data/points.static_pressure.data.resampled.h5"
)
body_data_path = pathlib.Path("./fixtures/tests/pressure/data/bodies.galpao.data.resampled.h5")

press_data: pd.DataFrame = pd.read_hdf(static_data_path)  # type: ignore
body_data: pd.DataFrame = pd.read_hdf(body_data_path)  # type: ignore

press_data, body_data

/tmp/ipykernel_35340/487441141.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


(      point_idx       rho  time_step
 2915          0  1.002515    10000.0
 2915          0  1.002532    10001.0
 2915          0  1.002549    10002.0
 2915          0  1.002566    10003.0
 2915          0  1.002583    10004.0
 ...         ...       ...        ...
 2915          0  1.003020    10096.0
 2915          0  1.003010    10097.0
 2915          0  1.002998    10098.0
 2915          0  1.002987    10099.0
 2915          0  1.002974    10100.0
 
 [101 rows x 3 columns],
       point_idx       rho  time_step
 0             0  1.001819    10000.0
 1             1  1.001838    10000.0
 2             2  1.001808    10000.0
 3             3  1.001851    10000.0
 4             4  1.001835    10000.0
 ...         ...       ...        ...
 2910       2910  1.002633    10100.0
 2911       2911  1.002594    10100.0
 2912       2912  1.002504    10100.0
 2913       2913  1.002828    10100.0
 2914       2914  1.002812    10100.0
 
 [294415 rows x 3 columns])

Process pressure coefficient use case

In [8]:
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.cp_data import process_cp
from cfdmod.use_cases.pressure.path_manager import copy_input_artifacts

for cfg_lbl, cfg in post_proc_cfg.pressure_coefficient.items():
    logger.info(f"Processing pressure coefficients for config {cfg_lbl} ...")

    cp_output = process_cp(
        pressure_data=press_data,
        body_data=body_data,
        cfg=cfg,
        mesh=mesh.geometry,
        extreme_params=post_proc_cfg.extreme_values,
    )
    cp_output.save_outputs(cfg=cfg, cfg_label=cfg_lbl, path_manager=path_manager)

    logger.info(f"Processed pressure coefficients for config {cfg_lbl}!")


[2024-01-30 16:01:26,707] [INFO] - cfdmod - Processing pressure coefficients for config default ... (1486552892.py:6)
[2024-01-30 16:01:28,097] [INFO] - cfdmod - Processed pressure coefficients for config default! (1486552892.py:17)


Copy Input artifacts to output folder

In [9]:
copy_input_artifacts(
    cfg_path=cfg_path,
    mesh_path=mesh_path,
    static_data_path=static_data_path,
    body_data_path=body_data_path,
    path_manager=path_manager,
)